In [6]:
import sys

message = 'Hello'
items = tuple(["Name","Ishe Doro","Course","Python Security","Hours",10])

print("Size of message: ", sys.getsizeof(message))
print("Size of message: ", sys.getsizeof(items))

Size of message:  54
Size of message:  88


In [7]:
print("Size of message hash: ", sys.getsizeof(hash(message)))
print("Size of list hash: ", sys.getsizeof(hash(items)))

Size of message hash:  36
Size of list hash:  36


In [38]:
class MyData:

    def __hash__(self):
        # Implement hash value generation

-4832963481255929395

In [20]:
# AVALANCHE EFFECT
print(abs(int(hex(0X02),16) - int(hex(0X03),16))) # Small diff
print(abs(hash(hex(0X02)) - hash(hex(0X03)))) # Large diff in hash value

1
4997587774213171717


In [31]:
class MyItem:

    def __init__(self,name):
        self.name = name
    
    def __hash__(self):
        return hash(self.name)

In [33]:
item = MyItem("Ishe")
item2 = MyItem("Ishe Doro")
hash(item) == hash(item2)

False

In [42]:
print(hash(-1))
print(hash(-2))

-2
-2


In [45]:
# HASHLIB

#Named constructor
import hashlib
import json
from hashlib import sha256

data = str.encode(json.dumps([{"Val":x, "Val Squared":x**2} for x in range(10)]))
sha256_obj = sha256()
sha256_obj_1 = sha256(bytes(data))

#Generic constructor
# Secure context
sha256_obj_generic = hashlib.new("sha256",bytes(data)) 
# Non-secure
sha256_obj_generic_nonsecure = hashlib.new("sha256",usedforsecurity=False) 

In [47]:
# Secure Hash As Bytes
sha256_obj_1.digest()

b'OX\x07\xcd\xa7\x11\xcb\t\x0c\xc1\xb1\xcf\xc8FZ\xbe\xe9\xe3x\xb1\xaf\xf7\xbby\x13\xd8\x02\x8ccM\xe0*'

In [48]:
# Secure Hash As Hexadecimal
sha256_obj_1.hexdigest()

'4f5807cda711cb090cc1b1cfc8465abee9e378b1aff7bb7913d8028c634de02a'

In [101]:
hash("Hello World")

5215509115238438366

In [107]:
message ="Hello World"
sha256_obj.update(str.encode(message))
sha256_obj.hexdigest()

'84aa7efe7a1e4c14b31adf2fe6960f2d3ea5de6dbdd8c31a291f9084f0544b27'

In [122]:
message ="Hello World"
sha256_obj.update(str.encode(message)) #update is called incrementally
sha256_obj.hexdigest() # Recomputes the secure hash for the concantenated message

'5244bdb0c035bf793d3b094bf314699514e23caa623217cc601a4981c79fd1ae'

In [121]:
sha256(str.encode(message)).hexdigest()

'a591a6d40bf420404a011733cfb7b190d62c65bf0bcda32b57b277d9ad9f146e'

In [90]:
# Incremental use of update
message_list =["Hello"," ","World"]
sha256_obj_a = sha256()
for m in message_list:
    sha256_obj_a.update(str.encode(m))
sha256_obj.hexdigest()

'5f7b224bf9b54877b3aa3a9b36c38b9c26e9746e418f878f8ca224c144d53fe6'

In [127]:
from hmac import compare_digest
def generate(data):
    # Incremental use of update
    sha256_obj = sha256(str.encode(data))
    return {"Data":data, "Hash":sha256_obj.hexdigest()}

def verify(message):
    data, secure_hash = message
    sha256_obj = sha256(message["Data"])
    if compare_digest(sha256_obj.hexdigest(),secure_hash):
        print("Message okay")
    else:
        print("Message corrupted")

In [129]:
message = generate("This is a test message")
verify(json.dumps(message))

ValueError: too many values to unpack (expected 2)

In [126]:
hashlib.algorithms_guaranteed

{'blake2b',
 'blake2s',
 'md5',
 'sha1',
 'sha224',
 'sha256',
 'sha384',
 'sha3_224',
 'sha3_256',
 'sha3_384',
 'sha3_512',
 'sha512',
 'shake_128',
 'shake_256'}

In [125]:
len(hashlib.algorithms_available)

22

In [ ]:
'AAAAAAAAAAAAAB7849' # Hexadecimal 16 possible 0 - 9 A - F
'AA0000000000000000' # N bytes long

In [5]:
import secrets
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

key = secrets.token_bytes(32)

cipher = Cipher(algorithms.AES(key),modes.ECB())

# encryption CipherContext instance
def encrypt_ecb(data):
    encryptor = cipher.encryptor()
    return encryptor.update(data) + encryptor.finalize()

# decrypt CipherContext instance
def encrypt_ecb(data):
    decryptor = cipher.decryptor()
    return decryptor.update(data) + decryptor.finalize()

In [11]:
import os
# Cipher Block Chaining Mode
def encrypt(data):
    iv = secrets.token_bytes(16)
    cipher = Cipher(algorithms.AES(key),modes.CBC(iv))
    encryptor = cipher.encryptor()
    return (encryptor.update(data) + encryptor.finalize(),iv)

def decrypt(ciphertext):
    ct,iv = ciphertext
    cipher = Cipher(algorithms.AES(key),modes.CBC(iv))
    decryptor = cipher.decryptor()
    return (decryptor.update(ct) + decryptor.finalize(),iv)

In [45]:
from cryptography.hazmat.primitives import padding

padder = padding.PKCS7(128).padder()
message = padder.update(b"Hello there") + padder.finalize()
ciphertext_1 = encrypt(message)
ciphertext_2 = encrypt(message)
print(ciphertext_1)
print(ciphertext_2)

(b'\x05M\xac\x98\xfd\xe4\xa0/\n\xd6\xbb\x85\xeeu\xc8\x83', b'6666666666666666')
(b'\x05M\xac\x98\xfd\xe4\xa0/\n\xd6\xbb\x85\xeeu\xc8\x83', b'6666666666666666')


In [46]:
message

b'Hello there\x05\x05\x05\x05\x05'

In [ ]:
# Galois Counter Mode (GCM) - Emulate a stream cipher

def authenticated_encrypt_gcm(plaintext,associated_data):
    iv = secrets.token_bytes(10)
    encryptor = Cipher(algorithms.AES(key),modes.GCM(iv)).encryptor()
    encryptor.authenticate_additional_data(associated_data)
    ciphertext = encryptor.update(plaintext) + encryptor.finalize()
    return (iv, ciphertext,encryptor.tag)

def decrypt_with_gcm(token,associated_data):
    iv, ciphertext,tag = token
    decryptor = Cipher(algorithms.AES(key),modes.GCM(iv,tag)).decryptor()
    decryptor.authenticate_additional_data(associated_data)
    return decryptor.update(ciphertext) + decryptor.finalize()
    

In [13]:
ciphertext,iv = encrypt(message)

In [34]:
cipher_file = open('cipher_file.tk','w')
cipher_file.writelines(ciphertext.hex() + " - " + iv.hex())
cipher_file.close()

In [37]:
cipher_file = open('cipher_file.tk','r')
token = cipher_file.readline()
cipher_file.close()

In [39]:
token.split(' - ')

['47b842bdeabd9cb465402e6ba0b37910', '36363636363636363636363636363636']

In [47]:
# RSA KEY GENERATION
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa

private_key = rsa.generate_private_key(public_exponent=65537,key_size=3072)
public_key = private_key.public_key()

In [50]:
private_bytes = private_key.private_bytes(encoding=serialization.Encoding.PEM,
format=serialization.PrivateFormat.PKCS8,encryption_algorithm=serialization.NoEncryption())

with open('my_private_key.pem','wb') as private_key_file:
    private_key_file.write(private_bytes)




In [61]:
public_bytes = public_key.public_bytes(encoding=serialization.Encoding.PEM,
format=serialization.PublicFormat.SubjectPublicKeyInfo)

with open('public_key_file.pem','wb') as public_key_file:
    public_key_file.write(public_bytes)

In [55]:
def load_key(is_private=True):
    if is_private:
        with open('my_private_key.pem','rb') as my_private_key:
            return serialization.load_pem_private_key(my_private_key.read(),password=None)
    else:
        with open('public_key_file.pem','rb') as public_key_file:
            return serialization.load_pem_public_key(public_key_file.read())

In [62]:
# RSA Encryption
# OAEP - Optional asymmetric encryption padding
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding

padding_config = padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()),algorithm=hashes.SHA256(),label=None)
public_key_from_disk = load_key(False)
ciphertext = public_key_from_disk.encrypt(plaintext=b"Sample data",padding=padding_config)

In [63]:
ciphertext

b'C>\xf9\xbeW\x87D\xf6\xef\x15*\x8a\x8e\x11!1\xe5\xf9|\x84_\xc6P\xec\x1a\x8c2\xb0<\x0fD\xb4(V\xa7%(\xd1\xc2L\xee\x83\x84\x9f\xf1\x17\x15\xe5s\x02k\xc0yC\r\xe5\xb5I\xdc\xe1\x14V1\xe3h"\xe1\x84\x0f\xa7$\xdfb\x82=\x03\x0ff]@M\xf6WL\xae\xa9s&c\xac\x13\xf2\x1c\xb3\x1fv\xd0\xbaS\xa6\x10i\xe8\xef\xf6a6\x1c\x9b\x93C\x95\xc4\x91~>\x94\xd7\xea\x07\xb9\xbd\xe6\xebqG\xba61\xad\xfb\xb2k\xdf%\xe0\xf6\xdf\xc7\xb7\xaa\xb7\x00\xa0\x08\xfd)$/\xeb\xec\xf3\xe7\xda\x1f\xfc\xa0W\xa9\xd2\xec\x01 \x11\xea\x18\x0e\x8b\xffg\xdfJ\x01h\x93eR \x17P\x03<\x8c\xcc\xc3/\x8d\xfa\xefI\x07~\x1b+<2\xf9\x03\x07>6\xe0\xf8\x94\xab\x1fS\xb4L\t\xbb\n\xe3\xb1J\x9f\xe2\xda\xe9\xa3\xe4)\x8d94\xadI\xf9^\xc1\xc4*\x08\xdf\x11\xfc\x84\x1f>a\x14\xed\xe6\xaa\x1aLX\x92\xc7\xaeUb\x1f\xce\xc0}\xfe\xa9\xdc\x1c\x04\xb1\x02\xdd.\xc4\xe4\xc0\n\x1co"Yz}\x8a]\xece\xf1^p\x11r\x10\xa1\x8ezB4u\xac\xa3\x803F4\xccw\xcb\xd6\xc6\xd7\xd8\x005\xa2\x05FnIC\x1dN\xc5\xfa\xf8\xf8\xe85rQ1\xc7\x9fBI\xa7\x88\xaf\x19\xe0\x11\xc2\xc0\xf4\xea\xa2\xe2\xfdZ\x84\x9b

In [64]:
# Decrypt with private key
load_key().decrypt(ciphertext=ciphertext,padding=padding_config)

b'Sample data'